In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale

In [2]:
def Listify_DataFrame(df, col_name):
    res = []
    for i in range(0, df.shape[0]):
        res.append(df[col_name][i])
    return res

In [10]:
Drug_List = ['Propoxyphene', 'Morphine', 'Methadone', 'Heroin',
       'Hydromorphone', 'Oxycodone', 'Oxymorphone', 'Dextropropoxyphene',
       'Buprenorphine', 'Fentanyl', 'Meperidine', 'Hydrocodone',
       'Dihydromorphone', 'Methorphan', 'Codeine', 'Tramadol', 'Pethidine',
       'Dihydrocodeine', 'Opiates', 'Opium', 'Pentazocine', 'Alphaprodine',
       'Acetylcodeine', 'Thebaine', 'Butorphanol', 'Nalbuphine', 'ANPP',
       'Desmethylprodine ', 'Remifentanil', 'Mitragynine',
       'Acetyldihydrocodeine', 'Hydrocodeinone', 'MT-45', 'Levorphanol',
       'Acetyl fentanyl', 'Butyryl fentanyl', 'Furanyl fentanyl',
       'p-Fluorobutyryl fentanyl', 'cis-3-methylfentanyl', 'Valeryl fentanyl',
       'Carfentanil', 'Acryl fentanyl', 'U-47700', '3-Methylfentanyl',
       '4-Fluoroisobutyryl fentanyl', 'trans-3-Methylfentanyl',
       'p-methoxybutyryl fentanyl', 'Tetrahydrofuran fentanyl',
       'p-Fluorofentanyl', 'o-Fluorofentanyl', '3-Fluorofentanyl',
       'Fluorofentanyl', 'Fluoroisobutyryl fentanyl',
       'Fluorobutyryl fentanyl ', 'Furanyl/3-Furanyl fentanyl',
       'Cyclopropyl fentanyl', 'Methoxyacetyl fentanyl', 'Phenyl fentanyl',
       'Benzylfentanyl', 'U-48800', 'U-49900', 'Crotonyl fentanyl',
       '3,4-Methylenedioxy U-47700', 'U-51754',
       'Cyclopropyl/Crotonyl Fentanyl', 'Isobutyryl fentanyl',
       'Cyclopentyl fentanyl', '4-Methylfentanyl', 'Metazocine']

In [4]:
len(Drug_List)

69

In [11]:
def score_county_with_cf(year):
    name='%d'%year
    year = year
    ct=pd.read_csv('%s.csv'%name)
    ct_np=np.array(ct)[0:,1:]
    
    ct.columns = ['County', 'Propoxyphene', 'Morphine', 'Methadone', 'Heroin',
       'Hydromorphone', 'Oxycodone', 'Oxymorphone', 'Dextropropoxyphene',
       'Buprenorphine', 'Hydrocodone', 'Meperidine', 'Fentanyl',
       'Dihydromorphone', 'Methorphan', 'Codeine', 'Tramadol', 'Pethidine',
       'Dihydrocodeine', 'Opiates', 'Opium', 'Pentazocine', 'Alphaprodine',
       'Acetylcodeine', 'Thebaine', 'Butorphanol', 'Nalbuphine', 'ANPP',
       'Desmethylprodine ', 'Remifentanil', 'Mitragynine',
       'Acetyldihydrocodeine', 'Hydrocodeinone', 'MT-45', 'Levorphanol',
       'Acetyl fentanyl', 'Butyryl fentanyl', 'Furanyl fentanyl',
       'p-Fluorobutyryl fentanyl', 'cis-3-methylfentanyl', 'Valeryl fentanyl',
       'Carfentanil', 'Acryl fentanyl', 'U-47700', '3-Methylfentanyl',
       '4-Fluoroisobutyryl fentanyl', 'trans-3-Methylfentanyl',
       'p-methoxybutyryl fentanyl', 'Tetrahydrofuran fentanyl',
       'p-Fluorofentanyl', 'o-Fluorofentanyl', '3-Fluorofentanyl',
       'Fluorofentanyl', 'Fluoroisobutyryl fentanyl',
       'Fluorobutyryl fentanyl ', 'Furanyl/3-Furanyl fentanyl',
       'Cyclopropyl fentanyl', 'Methoxyacetyl fentanyl', 'Phenyl fentanyl',
       'Benzylfentanyl', 'U-48800', 'U-49900', 'Crotonyl fentanyl',
       '3,4-Methylenedioxy U-47700', 'U-51754',
       'Cyclopropyl/Crotonyl Fentanyl', 'Isobutyryl fentanyl',
       'Cyclopentyl fentanyl', '4-Methylfentanyl', 'Metazocine']
    County = Listify_DataFrame(ct, 'County')
    County_List = ct.columns
    
    max = 0
    for i in range(2, len(County_List)):
        if ct.describe()[County_List[i]]['max'] > max:
            max = ct.describe()[County_List[i]]['max']
    ct_np_sc = ct_np/max
    
    cf = Cf(itr = 200, alpha=0.3)
    cf.fit(ct_np_sc)
    Point_drug = np.sqrt(max)*cf.year
    Point_County = np.sqrt(max)*cf.feature
    
    Profile = []

    for i in range(0, Point_County.shape[0]):
        Profile_1 = []
        Profile_1.append(County[i])
        Profile_1.append(year)
        for j in range(0, Point_County.shape[1]):
            Profile_1.append(Point_County[i][j])
        Profile.append(Profile_1)
    Profile = pd.DataFrame(Profile)
    Profile.columns = ['FIPS', 'Year', 'Score_County_1', 'Score_County_2', 'Score_County_3']
    pd.DataFrame(Profile).to_csv("Profile_County_%d.csv"%year)
    
    Profile = []

    for i in range(0, Point_drug.shape[0]):
        Profile_1 = []
        Profile_1.append(Drug_List[i])
        Profile_1.append(year)
        for j in range(0, Point_drug.shape[1]):
            Profile_1.append(Point_drug[i][j])
        Profile.append(Profile_1)
    Profile = pd.DataFrame(Profile)
    Profile.columns = ['Drug', 'Year', 'Score_Drug_1', 'Score_Drug_2', 'Score_Drug_3']
    pd.DataFrame(Profile).to_csv("Profile_Drug_%d.csv"%year)
    

In [9]:
score_county_with_cf(2010)

10988.387942398373
0
1.8303699026443094
1
1.3430191783667274
2
1.2417960612756427
3
1.1948154696264497
4
1.1601005884270954
5
1.126664136943793
6
1.0905344424648808
7
1.0504519635802592
8
1.0068311885317864
9
0.961240315456244
10
0.9157673947174224
11
0.8723270790750569
12
0.8321924440095958
13
0.7958994364258009
14
0.7634242167403607
15
0.7344373723901495
16
0.7085040420008701
17
0.6851967845775925
18
0.6641413310582216
19
0.6450252930534068
20
0.6275915298858684
21
0.6116276584203225
22
0.5969563956634963
23
0.5834279932241182
24
0.570914654799112
25
0.5593064567362706
26
0.5485083083096427
27
0.5384376092764617
28
0.5290223798061291
29
0.5201997233932308
30
0.5119145378214134
31
0.5041184212710649
32
0.4967687386056914
33
0.4898278227313761
34
0.4832622914539881
35
0.47704246360032243
36
0.47114186045337525
37
0.46553678033279905
38
0.4602059356779903
39
0.45513014336140795
40
0.4502920602051371
41
0.445675956800314
42
0.4412675237365288
43
0.4370537052374541
44
0.4330225559737545
4

In [12]:
for i in range(1,8):
    score_county_with_cf(2010+i)

10802.356787894356
0
1.9963516190572257
1
1.5132951094057048
2
1.405194607211048
3
1.3491710014412093
4
1.3048117445006462
5
1.2623013387476802
6
1.218974675453794
7
1.1747145631374076
8
1.1304284103062283
9
1.0872564679937287
10
1.0461268497182528
11
1.0075897126638371
12
0.9718447795597692
13
0.938851523132221
14
0.9084407690912845
15
0.8803937457635651
16
0.8544866784597205
17
0.830511069755415
18
0.8082802417994501
19
0.7876294170139458
20
0.7684133874239645
21
0.7505036857811002
22
0.733786007342959
23
0.7181580840450449
24
0.7035279940143445
25
0.6898128234642306
26
0.6769375938685842
27
0.6648343830120084
28
0.6534415870605209
29
0.6427032865182701
30
0.6325686907262008
31
0.622991643871635
32
0.613930181119714
33
0.6053461271955635
34
0.5972047321275827
35
0.5894743403407797
36
0.5821260901742334
37
0.5751336414059058
38
0.5684729286499878
39
0.5621219386465148
40
0.5560605095541021
41
0.5502701504235383
42
0.5447338790944947
43
0.5394360768304852
44
0.5343623580919548
45
0.529

0.4138154297774791
163
0.4133755233825043
164
0.4129412875868135
165
0.4125126143177892
166
0.4120893981923072
167
0.41167153643616683
168
0.41125892880628595
169
0.41085147751555284
170
0.4104490871602575
171
0.4100516646499654
172
0.4096591191397728
173
0.40927136196483277
174
0.4088883065770761
175
0.40850986848404175
176
0.40813596518973205
177
0.40776651613743015
178
0.4074014426543944
179
0.40704066789836524
180
0.4066841168058194
181
0.4063317160419072
182
0.40598339395200805
183
0.4056390805148509
184
0.4052987072971384
185
0.4049622074096222
186
0.4046295154645832
187
0.40430056753465504
188
0.4039753011129568
189
0.4036536550744711
190
0.403335569638653
191
0.40302098633318495
192
0.40270984795887965
193
0.40240209855566517
194
0.4020976833696194
195
0.4017965488210238
196
0.40149864247339745
197
0.4012039130034782
198
0.4009123101721146
199
11033.707516579114
0
2.8687363351351003
1
2.3500055612641484
2
2.20982165255356
3
2.1191430910601663
4
2.036826134794055
5
1.95355525863

0.35082654271447106
125
0.3498160366607794
126
0.3488228541950178
127
0.3478465703238388
128
0.34688677350303954
129
0.34594306511505607
130
0.3450150589705667
131
0.344102380832944
132
0.34320466796432214
133
0.3423215686921686
134
0.34145274199527
135
0.34059785710814133
136
0.339756593142898
137
0.3389286387277113
138
0.33811369166099603
139
0.33731145858053857
140
0.33652165464682104
141
0.33574400323983933
142
0.33497823566873297
143
0.33422409089361776
144
0.33348131525901276
145
0.3327496622383144
146
0.3320288921887695
147
0.33131877211645383
148
0.330619075450799
149
0.3299295818281859
150
0.3292500768842008
151
0.3285803520541573
152
0.3279202043814823
153
0.3272694363336192
154
0.326627855625109
155
0.3259952750475118
156
0.325371512305871
157
0.324756389861427
158
0.3241497347802973
159
0.3235513785878635
160
0.3229611571286212
161
0.3223789104312465
162
0.3218044825786516
163
0.321237721582826
164
0.32067847926425164
165
0.3201266111356949
166
0.31958197629020046
167
0.319

0.4916763855983644
87
0.49007333604044373
88
0.48850668403464725
89
0.4869752648680775
90
0.4854779633045192
91
0.4840137108881749
92
0.4825814834230578
93
0.4811802986154205
94
0.47980921386757514
95
0.47846732421231947
96
0.4771537603780076
97
0.47586768697502124
98
0.4746083007951069
99
0.4733748292156559
100
0.4721665287016108
101
0.4709826833982033
102
0.469822603808236
103
0.46868562554807736
104
0.4675711081769845
105
0.4664784340947206
106
0.46540700750286446
107
0.46435625342546816
108
0.4633256167851146
109
0.4623145615306415
110
0.4613225698131264
111
0.46034914120693576
112
0.4593937919728836
113
0.4584560543607657
114
0.4575354759487318
115
0.4566316190171217
116
0.4557440599545707
117
0.4548723886943699
118
0.45401620817916766
119
0.4531751338522691
120
0.4523487931738881
121
0.45153682516085086
122
0.45073887994832146
123
0.4499546183722504
124
0.44918371157131237
125
0.4484258406072027
126
0.4476806961022357
127
0.4469479778932495
128
0.44622739470089473
129
0.445518663

In [160]:
name='2010'
year = 2010
ct=pd.read_csv('%s.csv'%name)
ct_np=np.array(ct)[0:,1:]

In [114]:
ct

,Unnamed: 0,Propoxyphene,Morphine,Methadone,Heroin,Hydromorphone,Oxycodone,Oxymorphone,Dextropropoxyphene,Buprenorphine,...,U-48800,U-49900,Crotonyl fentanyl,"3,4-Methylenedioxy U-47700",U-51754,Cyclopropyl/Crotonyl Fentanyl,Isobutyryl fentanyl,Cyclopentyl fentanyl,4-Methylfentanyl,Metazocine
0,51001,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,39001,0,9,12,50,4,119,1,0,6,...,0,0,0,0,0,0,0,0,0,0
2,42001,0,0,2,14,1,29,1,0,2,...,0,0,0,0,0,0,0,0,0,0
3,51510,0,0,2,5,1,5,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,42003,10,20,34,2592,5,280,16,0,236,...,0,0,0,0,0,0,0,0,0,0
5,21003,0,0,4,0,0,15,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,51007,0,0,0,1,1,1,0,0,2,...,0,0,0,0,0,0,0,0,0,0
7,51013,0,6,2,41,3,38,0,0,8,...,0,0,0,0,0,0,0,0,0,0
8,42005,1,1,0,96,0,4,2,1,16,...,0,0,0,0,0,0,0,0,0,0
9,39005,0,6,2,82,0,45,2,0,12,...,0,0,0,0,0,0,0,0,0,0


In [98]:
ct.columns = ['County', 'Propoxyphene', 'Morphine', 'Methadone', 'Heroin',
       'Hydromorphone', 'Oxycodone', 'Oxymorphone', 'Dextropropoxyphene',
       'Buprenorphine', 'Hydrocodone', 'Meperidine', 'Fentanyl',
       'Dihydromorphone', 'Methorphan', 'Codeine', 'Tramadol', 'Pethidine',
       'Dihydrocodeine', 'Opiates', 'Opium', 'Pentazocine', 'Alphaprodine',
       'Acetylcodeine', 'Thebaine', 'Butorphanol', 'Nalbuphine', 'ANPP',
       'Desmethylprodine ', 'Remifentanil', 'Mitragynine',
       'Acetyldihydrocodeine', 'Hydrocodeinone', 'MT-45', 'Levorphanol',
       'Acetyl fentanyl', 'Butyryl fentanyl', 'Furanyl fentanyl',
       'p-Fluorobutyryl fentanyl', 'cis-3-methylfentanyl', 'Valeryl fentanyl',
       'Carfentanil', 'Acryl fentanyl', 'U-47700', '3-Methylfentanyl',
       '4-Fluoroisobutyryl fentanyl', 'trans-3-Methylfentanyl',
       'p-methoxybutyryl fentanyl', 'Tetrahydrofuran fentanyl',
       'p-Fluorofentanyl', 'o-Fluorofentanyl', '3-Fluorofentanyl',
       'Fluorofentanyl', 'Fluoroisobutyryl fentanyl',
       'Fluorobutyryl fentanyl ', 'Furanyl/3-Furanyl fentanyl',
       'Cyclopropyl fentanyl', 'Methoxyacetyl fentanyl', 'Phenyl fentanyl',
       'Benzylfentanyl', 'U-48800', 'U-49900', 'Crotonyl fentanyl',
       '3,4-Methylenedioxy U-47700', 'U-51754',
       'Cyclopropyl/Crotonyl Fentanyl', 'Isobutyryl fentanyl',
       'Cyclopentyl fentanyl', '4-Methylfentanyl', 'Metazocine']

In [107]:
County = Listify_DataFrame(ct, 'County')

In [108]:
print(County)

[51001, 39001, 42001, 51510, 42003, 21003, 51007, 51013, 42005, 39005, 39007, 39009, 39011, 54001, 21009, 21011, 42009, 51019, 21013, 54003, 42011, 51021, 21015, 54005, 21023, 54007, 21025, 21027, 39015, 51029, 42017, 21029, 39017, 42019, 54011, 54013, 42021, 42023, 42025, 51033, 21041, 39019, 21043, 21045, 51540, 51550, 42029, 51041, 21047, 42031, 42033, 39025, 39027, 51570, 42037, 39029, 39033, 42039, 21055, 39035, 51590, 42043, 21059, 39039, 39041, 42045, 51051, 54017, 39043, 42049, 51057, 51059, 21001, 51003, 51005, 39003, 51009, 51015, 51017, 42007, 39013, 21017, 21019, 21021, 51520, 51027, 51530, 21035, 21037, 51031, 51035, 42027, 39021, 21049, 51043, 21051, 21053, 42035, 39031, 51045, 51047, 21057, 51049, 39037, 51053, 21063, 51600, 39045, 21005, 51515, 42013, 51023, 42015, 51037, 42041, 42047, 51595, 21065, 54009, 21031, 21039, 39023, 54015, 21007, 51025, 21033, 51610, 21067, 21061, 51061, 51011, 39047, 42051, 21071, 51063, 51065, 21073, 39049, 42055, 51067, 51069, 21075, 42057

In [118]:
County_List = ct.columns
max = 0
for i in range(2, len(County_List)):
    if ct.describe()[County_List[i]]['max'] > max:
        max = ct.describe()[County_List[i]]['max']
ct_np_sc = ct_np/max
Ct_Np_Sc = pd.DataFrame(ct_np_sc)

In [119]:
ct_np_sc

array([[0.00025575, 0.00025575, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00230179, 0.00306905, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00051151, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [5]:
class Cf():
    def __init__(self,lbd=0.01,itr=100,cls=3,alpha=0.3): 
        self.lbd=lbd
        self.itr=itr 
        self.cls=cls 
        self.alpha=alpha 
        self.cost=np.zeros(itr) 
        self.year=[] 
        self.feature=[]
        
    def fit(self,x): 
        self.year=np.random.rand(x.shape[1],self.cls) 
#         print(self.year)
        self.feature= np.random.rand(x.shape[0],self.cls)
#         print(self.feature)
        
        for i in range(self.itr): 
            self.cost[i]=self.cost_fun(self.year,self.feature,x) 
            print(self.cost[i])
            print(i)
            if i > 0:
                if(self.cost[i] > self.cost[i-1]< 0.001):
                    return
            for ny in range(self.year.shape[0]):
                for nf in range(self.feature.shape[0]):
#                     print(ny, nf)
                    self.year[ny,:] -= self.alpha * (np.dot(self.feature[nf,:].transpose(),self.year[ny,:]) - x[nf,ny]) * self.year[ny,:]
            for nf in range(self.feature.shape[0]):
                for ny in range(self.year.shape[0]):
#                     print(ny, nf)
                    self.feature[nf,:]-=self.alpha * np.dot( np.dot(self.feature[nf,:].transpose(),self.year[ny,:]) - x[nf,ny] , self.feature[nf,:])
        
        
                                                                 
    def cost_fun(self,year,feature,x):
        y = np.dot(feature,year.transpose())
#         print(y)
#         print(x)
#         print(y-x)
        Cost  = (y-x)**2
        cost=1/2*sum(sum(Cost))
#         print(Cost)
#         print(cost)
        return cost

In [145]:
cf = Cf(itr = 500, alpha=0.3)

In [146]:
cf.cost

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [147]:
cf.fit(ct_np_sc)

29454.997215029245
0
1.7821100019357632
1
1.30937643249221
2
1.2050580402633664
3
1.1519069122634429
4
1.1107346492638537
5
1.0722228977416277
6
1.0338411152159752
7
0.9953060972816137
8
0.9571529958194142
9
0.9201023567651472
10
0.8847601953619587
11
0.851515932474081
12
0.8205522408790005
13
0.7918986639826991
14
0.7654875146142923
15
0.7411959792632961
16
0.7188733776382702
17
0.6983577513440831
18
0.67948597186023
19
0.6621000768850842
20
0.646051264454671
21
0.6312022595803681
22
0.6174284665744244
23
0.6046182159766055
24
0.5926723718684258
25
0.5815035270860727
26
0.5710349688073675
27
0.5611995498145278
28
0.551938557934293
29
0.5432006414828613
30
0.5349408228230269
31
0.5271196144088183
32
0.5197022402667789
33
0.5126579590166153
34
0.5059594808082758
35
0.49958246884183394
36
0.49350511566135496
37
0.4877077846547992
38
0.4821727078179081
39
0.4768837316485352
40
0.47182610390995355
41
0.46698629486951404
42
0.4623518474461781
43
0.4579112514676407
44
0.45365383793637337
45


0.29919684979955413
356
0.29914798219930994
357
0.2990994071754288
358
0.29905112213632523
359
0.2990031245208596
360
0.298955411797891
361
0.29890798146584197
362
0.29886083105226313
363
0.29881395811341466
364
0.29876736023384604
365
0.2987210350259922
366
0.2986749801297685
367
0.2986291932121755
368
0.29858367196691515
369
0.29853841411400595
370
0.29849341739941015
371
0.29844867959466376
372
0.2984041984965163
373
0.29835997192657415
374
0.2983159977309503
375
0.2982722737799203
376
0.29822879796758606
377
0.29818556821154063
378
0.29814258245254116
379
0.29809983865419226
380
0.2980573348026224
381
0.2980150689061783
382
0.2979730389951178
383
0.29793124312130836
384
0.29788967935793287
385
0.2978483457991966
386
0.29780724056004254
387
0.29776636177586996
388
0.2977257076022564
389
0.29768527621468566
390
0.2976450658082799
391
0.29760507459753854
392
0.29756530081607235
393
0.29752574271635523
394
0.2974863985694681
395
0.2974472666648558
396
0.2974083453100786
397
0.297369632

In [148]:
cf.year.shape

(69, 5)

In [149]:
cf.feature.shape

(461, 5)

In [150]:
Point_drug = np.sqrt(max)*cf.year
Point_County = np.sqrt(max)*cf.feature

In [171]:
Point_drug

array([[6.69440488e-03, 3.75501009e-03, 1.44765109e-03, 6.55627278e-03,
        2.83232200e-04],
       [1.21786956e-02, 2.40698484e-02, 2.09057814e-02, 2.46739837e-02,
        2.25971366e-02],
       [1.68214079e-02, 3.91557791e-02, 1.23374796e-02, 3.35944062e-02,
        1.88510992e-02],
       [1.03926375e-01, 5.78126289e-01, 1.34707214e-01, 7.11259079e-01,
        1.54134368e-01],
       [1.26869094e-02, 9.03408316e-03, 3.88819967e-04, 2.16102704e-02,
        2.75083368e-03],
       [1.96260267e-02, 5.61911974e-01, 3.15593137e-01, 3.57319198e-01,
        1.25251950e-01],
       [1.87340979e-03, 7.81307569e-03, 3.07860157e-03, 5.22156720e-03,
        6.23660704e-03],
       [8.32997005e-04, 8.82452180e-04, 6.35555183e-04, 7.13975783e-04,
        9.48854997e-04],
       [3.17660620e-03, 5.69285306e-02, 5.61395453e-02, 1.58418139e-02,
        5.57560607e-02],
       [6.63840931e-02, 2.15040010e-01, 1.67781273e-01, 6.80319881e-02,
        7.24984093e-02],
       [9.92803674e-04, 8.5062

In [152]:
np.dot(Point_County, Point_drug.transpose())

array([[1.04098577e-01, 5.28986156e-01, 5.95720178e-01, ...,
        1.11354907e-02, 1.16011117e-02, 1.14749107e-02],
       [1.18488802e+00, 5.81827458e+00, 7.50781760e+00, ...,
        9.01436615e-02, 1.07893304e-01, 1.08454621e-01],
       [2.23522247e-01, 1.39492787e+00, 1.40713689e+00, ...,
        3.05050010e-02, 2.73707577e-02, 2.80333753e-02],
       ...,
       [5.41966177e-02, 4.33075807e-01, 4.78485577e-01, ...,
        8.86404494e-03, 5.78543930e-03, 7.54687871e-03],
       [8.85187835e-02, 4.22133994e-01, 4.54779728e-01, ...,
        9.39070979e-03, 1.06460040e-02, 7.90493990e-03],
       [5.69085245e-02, 4.13996465e-01, 5.02498337e-01, ...,
        7.03917812e-03, 4.99406942e-03, 6.79798251e-03]])

In [161]:
Profile = []

for i in range(0, Point_County.shape[0]):
    Profile_1 = []
    Profile_1.append(County[i])
    Profile_1.append(year)
    for j in range(0, Point_County.shape[1]):
        Profile_1.append(Point_County[i][j])
    Profile.append(Profile_1)

In [162]:
Profile = pd.DataFrame(Profile)

In [163]:
Profile.columns = ['FIPS', 'Year', 'Score_County_1', 'Score_County_2', 'Score_County_3', 'Score_County_4', 'Score_County_5']

In [167]:
pd.DataFrame(Profile).to_csv("Profile_%d.csv"%year)

In [165]:
cost = cf.cost

,FIPS,Year,Score_County_1,Score_County_2,Score_County_3,Score_County_4,Score_County_5
0,51001,2010,7.364281,5.176366,7.766783,3.555117,2.859428
1,39001,2010,55.827220,68.907259,12.148996,79.140414,56.338600
2,42001,2010,17.791907,7.594808,20.078918,6.341383,18.551338
3,51510,2010,10.709370,2.217478,7.337846,7.287346,7.364550
4,42003,2010,840.192309,723.770348,271.955984,975.414750,884.899614
5,21003,2010,7.292971,8.456407,5.454217,6.661402,11.719638
6,51007,2010,0.677434,3.774051,6.349180,7.350467,2.091434
7,51013,2010,20.830571,28.518624,20.902292,25.500079,16.074734
8,42005,2010,38.305611,40.476520,2.856678,19.683683,34.249140
9,39005,2010,54.446305,12.218717,12.243194,81.128944,35.600153


In [182]:
cf.cost[0] = 3.6

In [185]:
from pyecharts import Line

attr = []
for i in range(0,len(cf.cost)):
    attr.append("%d"%i)
v1 = []
for i in range(0,len(cf.cost)):
    v1.append(cf.cost[i])
    
line = Line("Cost_Function_Iteration_Relation")
line.add("Cost", attr, v1, is_smooth=True, mark_line=["max", "average"] )
line.render()